In [1]:
from datetime import datetime
import os
import pickle
import random
import time
import warnings
from detector import Descriptor
import cv2
from skimage import feature
import numpy as np
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, fbeta_score, roc_auc_score, log_loss
from sklearn.metrics import confusion_matrix
import pandas as pd


In [2]:
__file__ = '/home/migtra/Escritorio/LEV_R/HOGSVM/migtra-migtra_hogsvm-00ea69232735/Inputs_Outputs/LEV_R/train'


In [3]:
base_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
pos_dir = __file__ + '/positives'
neg_dir = __file__ + '/negatives'

In [5]:
def processFiles(pos_dir, neg_dir, recurse=False, output_file=False,
        output_filename=None, color_space="bgr", channels=[0, 1, 2],
        hog_features=True, hist_features=False, spatial_features=False,
        hog_lib="cv", size=(64,64), hog_bins=9, pix_per_cell=(8,8),
        cells_per_block=(2,2), block_stride=None, block_norm="L1",
        transform_sqrt=True, signed_gradient=False, hist_bins=16,
        spatial_size=(16,16)):

    """
    Extract features from positive samples and negative samples.
    Store feature vectors in a dict and optionally save to pickle file.

    @param pos_dir (str): Path to directory containing positive samples.
    @param neg_dir (str): Path to directory containing negative samples.
    @param recurse (bool): Traverse directories recursively (else, top-level only).
    @param output_file (bool): Save processed samples to file.
    @param output_filename (str): Output file filename.
    @param color_space (str): Color space conversion.
    @param channels (list): Image channel indices to use.
    
    For remaining arguments, refer to Descriptor class:
    @see descriptor.Descriptor#__init__(...)

    @return feature_data (dict): Lists of sample features split into training,
        cross-validation, test sets; scaler object; parameters used to
        construct descriptor and process images.

    NOTE: OpenCV HOGDescriptor currently only supports 1-channel and 3-channel
    images, not 2-channel images.
    """

    if not (hog_features or hist_features or spatial_features):
        raise RuntimeError("No features selected (set hog_features=True, "
            + "hist_features=True, and/or spatial_features=True.)")

    pos_dir = os.path.abspath(pos_dir)
    neg_dir = os.path.abspath(neg_dir)

    if not os.path.isdir(pos_dir):
        raise FileNotFoundError("Directory " + pos_dir + " does not exist.")
    if not os.path.isdir(neg_dir):
        raise FileNotFoundError("Directory " + neg_dir + " does not exist.")

    print("Building file list...")
    if recurse:
        pos_files = [os.path.join(rootdir, file) for rootdir, _, files
            in os.walk(pos_dir) for file in files]
        neg_files = [os.path.join(rootdir, file) for rootdir, _, files
            in os.walk(neg_dir) for file in files]
    else:
        pos_files = [os.path.join(pos_dir, file) for file in
            os.listdir(pos_dir) if os.path.isfile(os.path.join(pos_dir, file))]
        neg_files = [os.path.join(neg_dir, file) for file in
            os.listdir(neg_dir) if os.path.isfile(os.path.join(neg_dir, file))]

    print("{} positive files and {} negative files found.\n".format(
        len(pos_files), len(neg_files)))

    # Get color space information.
    color_space = color_space.lower()
    if color_space == "gray":
        color_space_name = "grayscale"
        cv_color_const = cv2.COLOR_BGR2GRAY
        channels = [0]
    elif color_space == "hls":
        color_space_name = "HLS"
        cv_color_const = cv2.COLOR_BGR2HLS
    elif color_space == "hsv":
        color_space_name = "HSV"
        cv_color_const = cv2.COLOR_BGR2HSV
    elif color_space == "lab":
        color_space_name = "Lab"
        cv_color_const = cv2.COLOR_BGR2Lab
    elif color_space == "luv":
        color_space_name = "Luv"
        cv_color_const = cv2.COLOR_BGR2Luv
    elif color_space == "ycrcb" or color_space == "ycc":
        color_space_name = "YCrCb"
        cv_color_const = cv2.COLOR_BGR2YCrCb
    elif color_space == "yuv":
        color_space_name = "YUV"
        cv_color_const = cv2.COLOR_BGR2YUV
    else:
        color_space_name = "BGR"
        cv_color_const = -1

    # Get names of desired features.
    features = [feature_name for feature_name, feature_bool
        in zip(["HOG", "color histogram", "spatial"],
               [hog_features, hist_features, spatial_features])
        if feature_bool == True]

    feature_str  = features[0]
    for feature_name in features[1:]:
        feature_str += ", " + feature_name

    # Get information about channel indices.
    if len(channels) == 2 and hog_features and hog_lib == "cv":
        warnings.warn("OpenCV HOG does not support 2-channel images",
            RuntimeWarning)

    channel_index_str = str(channels[0])
    for ch_index in channels[1:]:
        channel_index_str += ", {}".format(ch_index)

    print("Converting images to " + color_space_name + " color space and " 
        + "extracting " + feature_str + " features from channel(s) "
        + channel_index_str + ".\n")

    # Store feature vectors for positive samples in list pos_features and
    # for negative samples in neg_features.
    pos_features = []
    neg_features = []
    start_time = time.time()

    # Get feature descriptor object to call on each sample.
    descriptor = Descriptor(hog_features=hog_features, hist_features=hist_features,
            spatial_features=spatial_features, hog_lib=hog_lib, size=size,
            hog_bins=hog_bins, pix_per_cell=pix_per_cell,
            cells_per_block=cells_per_block, block_stride=block_stride,
            block_norm=block_norm, transform_sqrt=transform_sqrt,
            signed_gradient=signed_gradient, hist_bins=hist_bins,
            spatial_size=spatial_size)

    # Iterate through files and extract features.
    for i, filepath in enumerate(pos_files + neg_files):
        image = cv2.imread(filepath)
        if cv_color_const > -1:
            image = cv2.cvtColor(image, cv_color_const)

        if len(image.shape) > 2:
            image = image[:,:,channels]

        feature_vector = descriptor.getFeatureVector(image)

        if i < len(pos_files):
            pos_features.append(feature_vector)
        else:
            neg_features.append(feature_vector)

    print("Features extracted from {} files in {:.1f} seconds\n".format(
        len(pos_features) + len(neg_features), time.time() - start_time))

    # Store the length of the feature vector produced by the descriptor.
    num_features = len(pos_features[0])

    # Instantiate scaler and scale features.
    print("Scaling features.\n")
    scaler = StandardScaler().fit(pos_features + neg_features)
    pos_features = scaler.transform(pos_features)
    neg_features = scaler.transform(neg_features)
    
    # Randomize lists of feature vectors. Split 75/20/5 into training,
    # cross-validation, and test sets.
    print("Shuffling samples into training, cross-validation, and test sets.\n")
    random.shuffle(pos_features)
    random.shuffle(neg_features)

    num_pos_train = int(round(0.75 * len(pos_features)))
    num_neg_train = num_pos_train
    num_pos_val = int(round(0.2 * len(pos_features)))
    num_neg_val = int(len(neg_features) - round(0.8 * len(pos_features)))

    pos_train = pos_features[0 : num_pos_train]
    neg_train = neg_features[0 : num_neg_train]

    pos_val = pos_features[num_pos_train : (num_pos_train + num_pos_val)]
    neg_val = neg_features[num_neg_train : (num_neg_train + num_neg_val)]

    pos_test = pos_features[(num_pos_train + num_pos_val):]
    neg_test = neg_features[(num_neg_train + num_neg_val):]

    print("{} samples in positive training set.".format(len(pos_train)))
    print("{} samples in positive cross-validation set.".format(len(pos_val)))
    print("{} samples in positive test set.".format(len(pos_test)))
    print("{} total positive samples.\n".format(len(pos_train) +
        len(pos_val) + len(pos_test)))

    print("{} samples in negative training set.".format(len(neg_train)))
    print("{} samples in negative cross-validation set.".format(len(neg_val)))
    print("{} samples in negative test set.".format(len(neg_test)))
    print("{} total negative samples.\n".format(len(neg_train) +
        len(neg_val) + len(neg_test)))

    # Store sample data and parameters in dict.
    # Descriptor class object seems to produce errors when unpickling and
    # has been commented out below. The descriptor will be re-instantiated
    # by the Detector object later.
    feature_data = {
                    "pos_train": pos_train,
                    "neg_train": neg_train,
                    "pos_val": pos_val,
                    "neg_val": neg_val,
                    "pos_test": pos_test,
                    "neg_test": neg_test,
                    #"descriptor": descriptor,
                    "scaler": scaler,
                    "hog_features": hog_features,
                    "hist_features": hist_features,
                    "spatial_features": spatial_features,
                    "color_space": color_space,
                    "cv_color_const": cv_color_const,
                    "channels": channels,
                    "hog_lib": hog_lib,
                    "size": size,
                    "hog_bins": hog_bins,
                    "pix_per_cell": pix_per_cell,
                    "cells_per_block": cells_per_block,
                    "block_stride": block_stride,
                    "block_norm": block_norm,
                    "transform_sqrt": transform_sqrt,
                    "signed_gradient": signed_gradient,
                    "hist_bins": hist_bins,
                    "spatial_size": spatial_size,
                    "num_features": num_features
                    }

    # Pickle to file if desired.
    if output_file:
        if output_filename is None:
            output_filename = (datetime.now().strftime("%Y%m%d%H%M")
                + "_data.pkl")

        try:
            os.mkdir(os.path.join(base_path, 'Inputs_Outputs', 'trainer_files'))
        except: pass
        pickle.dump(feature_data, open(os.path.join(base_path, 'Inputs_Outputs', 'trainer_files', output_filename), "wb"))
        print("Sample and parameter data saved to {}\n".format(output_filename))

    return feature_data

In [6]:
feature_data = processFiles(pos_dir, neg_dir, recurse=False, output_file=False,
        output_filename=None, color_space="gray", channels=[0, 1, 2],
        hog_features=True, hist_features=False, spatial_features=False,
        hog_lib="cv", size=(128,128), hog_bins=12, pix_per_cell=(16,16),
        cells_per_block=(1,1), block_stride=None, block_norm="L1",
        transform_sqrt=True, signed_gradient=False, hist_bins=16,
        spatial_size=(16,16))

Building file list...
1586 positive files and 4753 negative files found.

Converting images to grayscale color space and extracting HOG features from channel(s) 0.

Features extracted from 6339 files in 7.1 seconds

Scaling features.

Shuffling samples into training, cross-validation, and test sets.

1190 samples in positive training set.
317 samples in positive cross-validation set.
79 samples in positive test set.
1586 total positive samples.

1190 samples in negative training set.
3484 samples in negative cross-validation set.
79 samples in negative test set.
4753 total negative samples.



In [7]:
# Train classifier on training set.
pos_train = np.asarray(feature_data["pos_train"])
neg_train = np.asarray(feature_data["neg_train"])
pos_val = np.asarray(feature_data["pos_val"])
neg_val = np.asarray(feature_data["neg_val"])
pos_test = np.asarray(feature_data["pos_test"])
neg_test = np.asarray(feature_data["neg_test"])

train_set = np.vstack((pos_train, neg_train))
train_labels = np.concatenate(
    (np.ones(pos_train.shape[0],), np.zeros(neg_train.shape[0],)))


#Validacion

val_set = np.vstack((pos_val, neg_val))
val_labels = np.concatenate(
    (np.ones(pos_val.shape[0],), np.zeros(neg_val.shape[0],)))

#test
test_set = np.vstack((pos_test, neg_test))
test_labels = np.concatenate(
    (np.ones(pos_test.shape[0],), np.zeros(neg_test.shape[0],)))


In [8]:
data = np.vstack((pos_train, pos_val, pos_test, neg_train, neg_val, neg_test))
labels = np.concatenate(
    (np.ones(pos_train.shape[0]+pos_val.shape[0]+pos_test.shape[0] ,), np.zeros(neg_train.shape[0]+neg_val.shape[0]+neg_test.shape[0],)))

In [20]:

results=pd.DataFrame(columns=['kernel','c','train_time','recall val', 'recall test',
                                 'prec val', 'prec test', 'acc val', 'acc test', 'f1 val', 'f1 test', 'auc val', 'auc test', 'log val', 'log test'])

kernel= ['linear','rbf']
c = [10**(-6+i) for i in range(12)]

i=0



for k in kernel:
    for ce in c:
        clf = svm.SVC(kernel= k,C = ce)
        t1 = time.time()
        clf.fit(train_set,train_labels)
        t2= time.time() - t1
        y_pred_val = clf.predict(val_set)
        y_pred_test = clf.predict(test_set)

        #Calculamos las métricas
        #recall
        recall_val = recall_score(val_labels, y_pred_val, average='macro')
        recall_test = recall_score(test_labels, y_pred_test, average='macro')
        #precision
        prec_val = precision_score(val_labels, y_pred_val)
        prec_test = precision_score(test_labels, y_pred_test)
        #accuracy
        acc_val = accuracy_score(val_labels, y_pred_val)
        acc_test = accuracy_score(test_labels,y_pred_test)
        f1_val = f1_score(val_labels, y_pred_val)
        f1_test = f1_score(test_labels, y_pred_test)  
        results.loc[i]=[k,ce,t2,recall_val, recall_test, prec_val, prec_test, acc_val, acc_test, f1_val, f1_test]
        i+=1


KeyboardInterrupt: 

In [5]:
hog_param = {}

color_space = ['gray', 'R', 'BGR', 'hsv']
pixel_per_cel = [(8,8), (16,16)]
cells_per_block = [(1,1),(2,2)]
hog_bin = [9,12]

for cs in color_space:
    for pix in pixel_per_cel:
        for cell in cells_per_block:
            for bins in hog_bin:
                t1 = time.time()
                feature_data = processFiles(pos_dir, neg_dir, recurse=False, output_file=False,
                                                                                output_filename=None, color_space=cs, channels=[0, 1, 2],
                                                                                hog_features=True, hist_features=False, spatial_features=False,
                                                                                hog_lib="cv", size=(128,128), hog_bins=bins, pix_per_cell=pix,
                                                                                cells_per_block=cell, block_stride=None, block_norm="L1",
                                                                                transform_sqrt=True, signed_gradient=False, hist_bins=16,
                                                                                spatial_size=(16,16))
                t2 =  time.time()-t1                                                                
                hog_param[(cs, pix, cell, bins, t2)] = feature_data

NameError: name 'processFiles' is not defined

In [4]:
hog_param

NameError: name 'hog_param' is not defined

In [11]:
results=pd.DataFrame(columns=['kernel','c','g','train_time','recall val', 'recall test',
                                 'prec val', 'prec test', 'acc val', 'acc test', 'f1 val', 'f1 test', 'auc val', 'auc test', 'log val', 'log test'])

kernel = ['linear','rbf']
C = [10**(-3+i) for i in range(9)]
gama = ['scale','auto']



def model_svm():
    
    i=0
    for k in kernel:
        for ce in C:
            if k == 'linear':
                clf = svm.SVC(kernel= k,C = ce,probability = True)
                t1 = time.time()
                clf.fit(train_set,train_labels)

                t2= time.time() - t1
                y_pred_val = clf.predict(val_set)
                y_pred_test = clf.predict(test_set)

                prob_val = clf.predict_proba(val_set)
                prob_test = clf.predict_proba(test_set)

                #Calculamos las métricas
                #recall
                recall_val = recall_score(val_labels, y_pred_val, average='macro')
                recall_test = recall_score(test_labels, y_pred_test, average='macro')
                #precision
                prec_val = precision_score(val_labels, y_pred_val)
                prec_test = precision_score(test_labels, y_pred_test)
                #accuracy
                acc_val = accuracy_score(val_labels, y_pred_val)
                acc_test = accuracy_score(test_labels,y_pred_test)

                #F1-Score
                f1_val = f1_score(val_labels, y_pred_val)
                f1_test = f1_score(test_labels, y_pred_test)
                #F1-Score
                f2_val = fbeta_score(val_labels, y_pred_val, beta = 2.0)
                f2_test = fbeta_score(test_labels, y_pred_test, beta = 2.0)
                #AUC-Score
                auc_val = roc_auc_score(val_labels, y_pred_val)
                auc_test = roc_auc_score(test_labels, y_pred_test)


                #Log loss
                log_val = log_loss(val_labels, prob_val)
                log_test = log_loss(test_labels, prob_test)
                

                results.loc[i]=[k,ce,'-',t2, recall_val, recall_test, prec_val, prec_test, acc_val, acc_test, f1_val, f1_test, auc_val, auc_test, log_val, log_test]
                i+=1

                
            elif k == 'rbf':
                
                for g in gama:
                    clf = svm.SVC(kernel= k,C = ce,gamma=g,probability = True)
                    t1 = time.time()
                    clf.fit(train_set,train_labels)

                    t2= time.time() - t1
                    y_pred_val = clf.predict(val_set)
                    y_pred_test = clf.predict(test_set)


                    prob_val = clf.predict_proba(val_set)
                    prob_test = clf.predict_proba(test_set)

                    #Calculamos las métricas
                    #recall
                    recall_val = recall_score(val_labels, y_pred_val, average='macro')
                    recall_test = recall_score(test_labels, y_pred_test, average='macro')
                    #precision
                    prec_val = precision_score(val_labels, y_pred_val)
                    prec_test = precision_score(test_labels, y_pred_test)
                    #accuracy
                    acc_val = accuracy_score(val_labels, y_pred_val)
                    acc_test = accuracy_score(test_labels,y_pred_test)

                    #F1-Score
                    f1_val = f1_score(val_labels, y_pred_val)
                    f1_test = f1_score(test_labels, y_pred_test)
                    #F1-Score
                    f2_val = fbeta_score(val_labels, y_pred_val, beta = 2.0)
                    f2_test = fbeta_score(test_labels, y_pred_test, beta = 2.0)
                    #AUC-Score
                    auc_val = roc_auc_score(val_labels, y_pred_val)
                    auc_test = roc_auc_score(test_labels, y_pred_test)


                    #Log loss
                    log_val = log_loss(val_labels, prob_val)
                    log_test = log_loss(test_labels, prob_test)
                    

                    results.loc[i]=[k,ce,g,t2, recall_val, recall_test, prec_val, prec_test, acc_val, acc_test, f1_val, f1_test, auc_val, auc_test, log_val, log_test]
                    i+=1
                        
                        
            


                

In [19]:
results

,kernel,c,g,train_time,recall val,recall test,prec val,prec test,acc val,acc test,f1 val,f1 test,auc val,auc test,log val,log test
0,linear,0.000001,-,35.267348,0.954259,0.949367,1.000000,1.000000,0.992370,0.949367,0.952066,0.946667,0.954259,0.949367,0.050001,0.153468
1,linear,0.000010,-,14.566532,0.987382,0.968354,1.000000,1.000000,0.997895,0.968354,0.987220,0.967320,0.987382,0.968354,0.015968,0.046042
2,linear,0.000100,-,4.609539,0.998423,1.000000,1.000000,1.000000,0.999737,1.000000,0.998420,1.000000,0.998423,1.000000,0.009207,0.023360
3,linear,0.001000,-,2.648127,0.998423,0.993671,1.000000,0.987500,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.005372,0.010136
4,linear,0.010000,-,2.501064,0.998423,0.993671,1.000000,0.987500,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.007047,0.013934
5,linear,0.100000,-,2.608155,0.998423,0.993671,1.000000,0.987500,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.005654,0.010440
6,linear,1.000000,-,2.444412,0.998423,0.993671,1.000000,0.987500,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.006984,0.011382
7,linear,10.000000,-,2.425357,0.998423,0.993671,1.000000,0.987500,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.004633,0.008567
8,linear,100.000000,-,2.414929,0.998423,0.993671,1.000000,0.987500,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.007024,0.011511
9,linear,1000.000000,-,2.426356,0.998423,0.993671,1.000000,0.987500,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.005497,0.010283


In [12]:
results[results['kernel']=='linear']

,kernel,c,g,train_time,recall val,recall test,prec val,prec test,acc val,acc test,f1 val,f1 test,auc val,auc test,log val,log test
0,linear,0.000001,-,35.267348,0.954259,0.949367,1.0,1.0000,0.992370,0.949367,0.952066,0.946667,0.954259,0.949367,0.050001,0.153468
1,linear,0.000010,-,14.566532,0.987382,0.968354,1.0,1.0000,0.997895,0.968354,0.987220,0.967320,0.987382,0.968354,0.015968,0.046042
2,linear,0.000100,-,4.609539,0.998423,1.000000,1.0,1.0000,0.999737,1.000000,0.998420,1.000000,0.998423,1.000000,0.009207,0.023360
3,linear,0.001000,-,2.648127,0.998423,0.993671,1.0,0.9875,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.005372,0.010136
4,linear,0.010000,-,2.501064,0.998423,0.993671,1.0,0.9875,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.007047,0.013934
5,linear,0.100000,-,2.608155,0.998423,0.993671,1.0,0.9875,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.005654,0.010440
6,linear,1.000000,-,2.444412,0.998423,0.993671,1.0,0.9875,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.006984,0.011382
7,linear,10.000000,-,2.425357,0.998423,0.993671,1.0,0.9875,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.004633,0.008567
8,linear,100.000000,-,2.414929,0.998423,0.993671,1.0,0.9875,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.007024,0.011511
9,linear,1000.000000,-,2.426356,0.998423,0.993671,1.0,0.9875,0.999737,0.993671,0.998420,0.993711,0.998423,0.993671,0.005497,0.010283


In [13]:
results[results['kernel']=='rbf']

,kernel,c,g,train_time,recall val,recall test,prec val,prec test,acc val,acc test,f1 val,f1 test,auc val,auc test,log val,log test
12,rbf,0.000001,scale,38.486241,0.959563,0.955696,0.976589,1.000000,0.991581,0.955696,0.948052,0.953642,0.959563,0.955696,0.693147,0.693147
13,rbf,0.000001,auto,39.205578,0.959276,0.949367,0.970100,0.986301,0.991055,0.949367,0.944984,0.947368,0.959276,0.949367,0.714437,0.693467
14,rbf,0.000010,scale,39.750853,0.959563,0.955696,0.976589,1.000000,0.991581,0.955696,0.948052,0.953642,0.959563,0.955696,0.693147,0.693147
15,rbf,0.000010,auto,41.456146,0.959276,0.949367,0.970100,0.986301,0.991055,0.949367,0.944984,0.947368,0.959276,0.949367,0.693147,0.693147
16,rbf,0.000100,scale,42.300059,0.959563,0.955696,0.976589,1.000000,0.991581,0.955696,0.948052,0.953642,0.959563,0.955696,0.693147,0.693147
17,rbf,0.000100,auto,42.202410,0.959276,0.949367,0.970100,0.986301,0.991055,0.949367,0.944984,0.947368,0.959276,0.949367,0.693147,0.693147
18,rbf,0.001000,scale,44.125961,0.959563,0.955696,0.976589,1.000000,0.991581,0.955696,0.948052,0.953642,0.959563,0.955696,0.693147,0.693147
19,rbf,0.001000,auto,41.334868,0.959276,0.949367,0.970100,0.986301,0.991055,0.949367,0.944984,0.947368,0.959276,0.949367,0.693147,0.693147
20,rbf,0.010000,scale,41.147850,0.965872,0.955696,0.976898,1.000000,0.992634,0.955696,0.954839,0.953642,0.965872,0.955696,0.078624,0.157482
21,rbf,0.010000,auto,40.492041,0.971607,0.949367,0.964630,0.986301,0.992634,0.949367,0.955414,0.947368,0.971607,0.949367,0.081490,0.161164


In [21]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score


clf = svm.SVC(kernel='linear', C=1, random_state=42,class_weight = { 0:0.75, 1:0.25 })
scores = cross_validate(clf, data, labels, cv=10,scoring=('recall','precision','accuracy','f1'),return_train_score=True)

In [27]:
scores['test_recall']

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.99367089, 1.        ])